# Fase 02: Comparación Dataset Original vs Cleaned
# Turkish Music Emotion Dataset

---

## Objetivo

Este notebook tiene como objetivo:
1. **Comparar** el dataset original con el dataset limpio
2. **Documentar métricas** de diferencias entre ambas versiones
3. **Visualizar** las transformaciones realizadas
4. **Preparar** los datos para versionado con DVC
5. **Analizar** el impacto de las transformaciones de limpieza

---

In [ ]:
# ==============================================================================
# SECCIÓN 1: CONFIGURACIÓN INICIAL Y LIBRERÍAS ESTÁNDAR
# ==============================================================================
import os
import sys
import warnings
import subprocess
from datetime import datetime
from pathlib import Path

# Configuración del Entorno
warnings.filterwarnings('ignore')

# ==============================================================================
# SECCIÓN 2: LIBRERÍAS DE TERCEROS PARA ANÁLISIS Y VISUALIZACIÓN
# ==============================================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import zscore

# ==============================================================================
# SECCIÓN 3: MÓDULOS Y FUNCIONES LOCALES DEL PROYECTO
# ==============================================================================
from acoustic_ml import (
    # Funciones para cargar los datasets
    load_turkish_modified,
    load_turkish_original,
    # Función para obtener información resumida del dataset
    get_dataset_info,
    # Función para guardar el dataset procesado
    save_processed_data,
    # Variables globales y constantes del proyecto
    RAW_DATA_DIR,
    PROCESSED_DATA_DIR,
    RANDOM_STATE
)

# ==============================================================================
# SECCIÓN 4: CONFIGURACIÓN DE VISUALIZACIÓN Y PANDAS
# ==============================================================================
# Configuración de Estilo para Gráficos
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# Configuración de Visualización de Pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: f'{x:.3f}')

# ==============================================================================
# SECCIÓN 5: VERIFICACIÓN FINAL
# ==============================================================================
print("✅ Librerías y configuración cargadas correctamente.")
print(f"📁 Directorio de datos RAW: {RAW_DATA_DIR}")
print(f"📁 Directorio de datos PROCESSED: {PROCESSED_DATA_DIR}")
print(f"🎲 Semilla aleatoria (Random State) global: {RANDOM_STATE}")

# Validacion de Entorno

In [ ]:
"""
================================================================================
🔍 VALIDACIÓN DEL ENTORNO Y DEPENDENCIAS
================================================================================
"""
import sys
import os
from pathlib import Path

# ====================
# 1. CONFIGURACIÓN DEL DIRECTORIO
# ====================
print("📁 Verificando directorio de trabajo...")
# Obtener directorio actual
current_dir = Path.cwd()
print(f"   Directorio actual: {current_dir}")

# Si estamos en notebooks/, subir un nivel
if current_dir.name == "notebooks":
    project_root = current_dir.parent
    os.chdir(project_root)
    print(f"   ✅ Raíz del proyecto: {project_root}")
else:
    project_root = current_dir
    print(f"   ✅ Ya estamos en la raíz: {project_root}")

print(f"   ✅ Directorio de trabajo: {Path.cwd()}\n")

# ====================
# 2. VERIFICAR MÓDULO ACOUSTIC_ML
# ====================
print("📦 Verificando módulo acoustic_ml...")
try:
    import acoustic_ml
    print(f"   ✅ acoustic_ml v{acoustic_ml.__version__}\n")
except ImportError as e:
    print(f"   ❌ ERROR: {e}")
    print("   💡 Solución: pip install -e .\n")
    sys.exit(1)

# ====================
# 3. VERIFICAR DEPENDENCIAS
# ====================
print("🔧 Verificando dependencias...")
dependencies = {
    'pandas': 'Procesamiento de datos',
    'numpy': 'Operaciones numéricas',
    'matplotlib': 'Visualizaciones',
    'seaborn': 'Visualizaciones estadísticas',
    'scipy': 'Tests estadísticos (KS test)'  # ← AGREGADO para KS test
}

missing_deps = []
for package, description in dependencies.items():
    try:
        __import__(package)
        print(f"   ✅ {package} - {description}")
    except ImportError:
        print(f"   ❌ {package} - {description}")
        missing_deps.append(package)

if missing_deps:
    print(f"\n   ❌ Faltan: {', '.join(missing_deps)}")
    print(f"   💡 Solución: pip install {' '.join(missing_deps)}\n")
    sys.exit(1)

print()

# ====================
# 4. VERIFICAR DVC
# ====================
print("☁️  Verificando estado de DVC...")
import subprocess

try:
    result = subprocess.run(
        ['dvc', 'status'],
        capture_output=True,
        text=True,
        cwd=Path.cwd(),
        timeout=10
    )
    
    if result.returncode == 0:
        print("   ✅ Datos sincronizados con S3\n")
    else:
        print(f"   ⚠️  DVC status: {result.stderr.strip()}\n")
except Exception as e:
    print(f"   ⚠️  No se pudo verificar DVC: {e}\n")

# ====================
# 5. VERIFICAR ARCHIVOS DE DATOS
# ====================
print("📂 Verificando archivos de datos necesarios para comparación...")

# Archivos específicos para este notebook de comparación
data_files = {
    'Original Dataset': Path('data/raw/turkis_music_emotion_original.csv'),
    'Modified Dataset': Path('data/raw/turkish_music_emotion_modified.csv'),
}

errors = []
for name, path in data_files.items():
    full_path = Path.cwd() / path
    if full_path.exists():
        size_kb = full_path.stat().st_size / 1024
        print(f"   ✅ {name} ({size_kb:.1f} KB)")
    else:
        print(f"   ❌ {name} - NO ENCONTRADO")
        print(f"      Ruta esperada: {full_path}")
        errors.append(f"Falta {name}")

print()

# ====================
# 6. VERIFICAR DIRECTORIOS DE SALIDA
# ====================
print("📁 Verificando/Creando directorios de salida...")

output_dirs = {
    'data/processed': 'Para datasets exportados',
    'reports': 'Para reportes de comparación'
}

for dir_path, description in output_dirs.items():
    full_path = Path.cwd() / dir_path
    if full_path.exists():
        print(f"   ✅ {dir_path}/ - {description}")
    else:
        full_path.mkdir(parents=True, exist_ok=True)
        print(f"   ✨ {dir_path}/ - Creado ({description})")

print()

# ====================
# 7. RESUMEN FINAL
# ====================
print("=" * 80)
if errors:
    print("❌ ERRORES DETECTADOS:")
    for i, error in enumerate(errors, 1):
        print(f"   {i}. {error}")
    print("\n💡 SOLUCIONES:")
    print("   • Datos faltantes: dvc pull")
    print("   • O verifica que los archivos estén en data/raw/")
    print("\n🛑 Deteniendo ejecución del notebook")
    print("=" * 80)
    # DETENER EJECUCIÓN
    raise RuntimeError("Validación falló - corrige los errores antes de continuar")
else:
    print("✅ TODAS LAS VALIDACIONES PASARON")
    print("🚀 El notebook de comparación está listo para ejecutarse")
    print("\n📋 Siguiente paso:")
    print("   • El notebook cargará los datasets original y cleaned")
    print("   • Realizará comparaciones exhaustivas")
    print("   • Exportará 3 versiones para DVC")
    print("   • Generará reporte en reports/")
    print("=" * 80)

---
## 1. Carga de Datos
---

In [ ]:
# ==============================================================================
# DESCARGA: Sincronizar datos desde S3 con DVC
# ==============================================================================
print("\n" + "="*80)
print("📥 SINCRONIZANDO DATOS DESDE S3 CON DVC")
print("="*80)

# Verificar si data/README.md existe y no está trackeado
data_readme = Path.cwd() / 'data' / 'README.md'
use_force = False

if data_readme.exists():
    print("📝 Detectado data/README.md (archivo de documentación)")
    print("   Este archivo NO debe ser versionado con DVC, solo con Git")
    print("   Usando --force para evitar conflictos...\n")
    use_force = True

try:
    # Ejecutar DVC pull (con --force si es necesario)
    cmd = ['dvc', 'pull']
    if use_force:
        cmd.append('--force')
    
    result = subprocess.run(
        cmd,
        check=True,
        capture_output=True,
        text=True,
        cwd=Path.cwd()
    )
    
    if result.stdout:
        print(result.stdout)
    
    print("✅ Datos sincronizados correctamente desde S3")
    print(f"📂 Ubicación: {RAW_DATA_DIR}")
    
    if use_force:
        print("\n💡 NOTA: Se usó --force para preservar data/README.md")
        print("   Recuerda commitear este archivo a Git después:")
        print("   git add data/README.md")
        print("   git commit -m 'docs: add data directory documentation'")
    
except subprocess.CalledProcessError as e:
    error_msg = e.stderr if e.stderr else str(e)
    print("⚠️  Error al ejecutar DVC pull:")
    print(error_msg)
    print("\n💡 SOLUCIONES POSIBLES:")
    print("   1. Verifica AWS credentials: aws configure")
    print("   2. Verifica conexión S3: aws s3 ls s3://mlops24-haowei-bucket/")
    print("   3. Verifica configuración DVC: dvc remote list")
    sys.exit(1)
    
except FileNotFoundError:
    print("❌ DVC no está instalado en el sistema")
    print("💡 Instala DVC con: pip install 'dvc[s3]'")
    sys.exit(1)

print("\n" + "="*80)

In [ ]:
# ==============================================================================
# CARGA: Dataset Original (sin modificaciones)
# ==============================================================================
print("\n" + "="*80)
print("📊 CARGANDO DATASET ORIGINAL")
print("="*80)

try:
    # Cargar dataset original usando el módulo acoustic_ml
    df_original = load_turkish_original()
    
    print(f"✅ Dataset original cargado exitosamente")
    print(f"📏 Shape: {df_original.shape[0]} filas × {df_original.shape[1]} columnas")
    
    # Mostrar información detallada del dataset
    print("\n" + "-"*80)
    print("INFORMACIÓN DETALLADA DEL DATASET ORIGINAL:")
    print("-"*80)
    get_dataset_info(df_original)
    
except FileNotFoundError as e:
    print(f"❌ Error: No se pudo encontrar el archivo del dataset original")
    print(f"   {str(e)}")
    print("\n💡 Verifica que DVC pull se haya ejecutado correctamente")
    sys.exit(1)
    
except Exception as e:
    print(f"❌ Error inesperado al cargar el dataset original: {str(e)}")
    sys.exit(1)

print("\n" + "="*80)

In [ ]:
# ==============================================================================
# CARGA: Dataset Modificado/Cleaned
# ==============================================================================
print("\n" + "="*80)
print("📊 CARGANDO DATASET MODIFICADO (CLEANED)")
print("="*80)

try:
    # Cargar dataset modificado usando el módulo acoustic_ml
    df_cleaned = load_turkish_modified()
    
    print(f"✅ Dataset modificado cargado exitosamente")
    print(f"📏 Shape: {df_cleaned.shape[0]} filas × {df_cleaned.shape[1]} columnas")
    
    # Mostrar información detallada del dataset
    print("\n" + "-"*80)
    print("INFORMACIÓN DETALLADA DEL DATASET MODIFICADO:")
    print("-"*80)
    get_dataset_info(df_cleaned)
    
except FileNotFoundError as e:
    print(f"❌ Error: No se pudo encontrar el archivo del dataset modificado")
    print(f"   {str(e)}")
    print("\n💡 Verifica que DVC pull se haya ejecutado correctamente")
    sys.exit(1)
    
except Exception as e:
    print(f"❌ Error inesperado al cargar el dataset modificado: {str(e)}")
    sys.exit(1)

print("\n" + "="*80)

---
## 2. Información General de los Datasets
---

In [ ]:
# ==============================================================================
# RESUMEN INICIAL: Comparación de Dimensiones
# ==============================================================================
print("\n" + "="*80)
print("📊 RESUMEN INICIAL DE COMPARACIÓN")
print("="*80)

print(f"\n{'Métrica':<30} {'Original':<15} {'Cleaned':<15} {'Diferencia':<15}")
print("-"*75)

# Comparar número de filas
diff_rows = df_cleaned.shape[0] - df_original.shape[0]
print(f"{'Número de Filas':<30} {df_original.shape[0]:<15} {df_cleaned.shape[0]:<15} {diff_rows:+<15}")

# Comparar número de columnas
diff_cols = df_cleaned.shape[1] - df_original.shape[1]
print(f"{'Número de Columnas':<30} {df_original.shape[1]:<15} {df_cleaned.shape[1]:<15} {diff_cols:+<15}")

# Comparar valores nulos
nulls_orig = df_original.isnull().sum().sum()
nulls_clean = df_cleaned.isnull().sum().sum()
diff_nulls = nulls_clean - nulls_orig
print(f"{'Valores Nulos Totales':<30} {nulls_orig:<15} {nulls_clean:<15} {diff_nulls:+<15}")

# Comparar duplicados
dups_orig = df_original.duplicated().sum()
dups_clean = df_cleaned.duplicated().sum()
diff_dups = dups_clean - dups_orig
print(f"{'Filas Duplicadas':<30} {dups_orig:<15} {dups_clean:<15} {diff_dups:+<15}")

# Comparar clases
classes_orig = df_original['Class'].nunique()
classes_clean = df_cleaned['Class'].nunique()
diff_classes = classes_clean - classes_orig
print(f"{'Número de Clases':<30} {classes_orig:<15} {classes_clean:<15} {diff_classes:+<15}")

print("\n" + "="*80)

# Mostrar warning si las dimensiones son diferentes
if diff_rows != 0:
    print("\n⚠️  ADVERTENCIA: Los datasets tienen diferente número de filas")
    print(f"   Se procederá a alinearlos para comparaciones directas")
    print(f"   Se usarán las primeras {min(len(df_original), len(df_cleaned))} filas comunes")

In [ ]:
# ==============================================================================
# VISTA PREVIA: Primeras Filas de Ambos Datasets
# ==============================================================================
print("\n" + "="*80)
print("👀 VISTA PREVIA DE LOS DATASETS")
print("="*80)

print("\n📋 PRIMERAS 5 FILAS - DATASET ORIGINAL:")
print("-"*80)
display(df_original.head())

print("\n📋 PRIMERAS 5 FILAS - DATASET CLEANED:")
print("-"*80)
display(df_cleaned.head())

# Verificar si las columnas son las mismas
cols_orig = set(df_original.columns)
cols_clean = set(df_cleaned.columns)

if cols_orig == cols_clean:
    print("\n✅ Ambos datasets tienen las mismas columnas")
else:
    print("\n⚠️  Los datasets tienen columnas diferentes:")
    
    only_orig = cols_orig - cols_clean
    if only_orig:
        print(f"\n   Solo en Original: {only_orig}")
    
    only_clean = cols_clean - cols_orig
    if only_clean:
        print(f"   Solo en Cleaned: {only_clean}")

print("\n" + "="*80)

# 2.5 Limpieza de Columnas Numericas

In [ ]:
# ==============================================================================
# IDENTIFICACIÓN INICIAL DE COLUMNAS NUMÉRICAS
# ==============================================================================
print("="*80)
print("🔍 IDENTIFICANDO COLUMNAS NUMÉRICAS INICIALES")
print("="*80)

# Obtener todas las columnas excepto 'Class'
columnas_para_analizar = [col for col in df_original.columns if col != 'Class']

print(f"\n📊 Total de columnas a analizar: {len(columnas_para_analizar)}")
print(f"   (Excluyendo la columna 'Class')")

# Identificar columnas que DEBERÍAN ser numéricas basándonos en el nombre del dataset
# (asumimos que todas excepto 'Class' son features numéricas)
columnas_numericas = columnas_para_analizar.copy()

print(f"\n✅ Columnas identificadas para limpieza: {len(columnas_numericas)}")
print("\n" + "="*80)

In [ ]:
# ==============================================================================
# LIMPIEZA Y VALIDACIÓN DE COLUMNAS NUMÉRICAS
# ==============================================================================
print("="*80)
print("🧹 LIMPIEZA Y VALIDACIÓN DE COLUMNAS NUMÉRICAS")
print("="*80)

def limpiar_columnas_numericas(df, columnas, nombre_dataset="Dataset"):
    """
    Limpia y valida columnas numéricas, convirtiendo valores no numéricos a NaN.
    
    Parameters:
    -----------
    df : DataFrame
        DataFrame a limpiar
    columnas : list
        Lista de columnas a validar como numéricas
    nombre_dataset : str
        Nombre del dataset para mensajes informativos
    
    Returns:
    --------
    df_limpio : DataFrame
        DataFrame con columnas convertidas a numérico
    columnas_limpias : list
        Lista de columnas que pasaron la validación
    """
    df_limpio = df.copy()
    columnas_limpias = []
    columnas_problematicas = []
    
    print(f"\n🔄 Procesando {nombre_dataset}...")
    print("-"*80)
    
    for col in columnas:
        if col in df.columns:
            # Guardar tipo original
            tipo_original = df[col].dtype
            
            # Convertir a numérico, convirtiendo errores a NaN
            df_limpio[col] = pd.to_numeric(df_limpio[col], errors='coerce')
            
            # Verificar cuántos valores se pudieron convertir
            n_validos = df_limpio[col].notna().sum()
            n_total = len(df_limpio)
            n_nulos_originales = df[col].isnull().sum()
            n_convertidos_nan = df_limpio[col].isnull().sum() - n_nulos_originales
            porcentaje_validos = (n_validos / n_total) * 100
            
            if porcentaje_validos >= 95:
                columnas_limpias.append(col)
                if n_convertidos_nan > 0:
                    print(f"  ⚠️  {col}: {n_convertidos_nan} valores no numéricos convertidos a NaN")
            else:
                columnas_problematicas.append({
                    'columna': col,
                    'tipo_original': tipo_original,
                    'pct_validos': porcentaje_validos,
                    'valores_invalidos': n_total - n_validos
                })
                print(f"  ❌ {col}: Solo {porcentaje_validos:.1f}% valores válidos - OMITIDA")
        else:
            print(f"  ⚠️  '{col}' no existe en el dataset")
    
    if columnas_problematicas:
        print(f"\n⚠️  {len(columnas_problematicas)} columnas omitidas por tener muchos valores no numéricos")
    else:
        print(f"\n✅ Todas las columnas pasaron la validación")
    
    return df_limpio, columnas_limpias, columnas_problematicas

# Limpiar ambos datasets
df_original_clean, cols_valid_orig, cols_prob_orig = limpiar_columnas_numericas(
    df_original, columnas_numericas, "Dataset Original"
)

df_cleaned_clean, cols_valid_clean, cols_prob_clean = limpiar_columnas_numericas(
    df_cleaned, columnas_numericas, "Dataset Cleaned"
)

# Obtener columnas válidas en ambos datasets
columnas_numericas_validas = list(set(cols_valid_orig) & set(cols_valid_clean))
columnas_numericas_validas.sort()

# Identificar columnas que están en uno pero no en otro
solo_en_original = set(cols_valid_orig) - set(cols_valid_clean)
solo_en_cleaned = set(cols_valid_clean) - set(cols_valid_orig)

print("\n" + "="*80)
print("📊 RESUMEN DE LIMPIEZA Y VALIDACIÓN")
print("="*80)
print(f"\n📈 ESTADÍSTICAS:")
print(f"   • Columnas iniciales:                {len(columnas_numericas)}")
print(f"   • Columnas válidas en Original:      {len(cols_valid_orig)}")
print(f"   • Columnas válidas en Cleaned:       {len(cols_valid_clean)}")
print(f"   • Columnas válidas COMUNES:          {len(columnas_numericas_validas)}")

if solo_en_original:
    print(f"\n⚠️  Columnas válidas SOLO en Original ({len(solo_en_original)}):")
    for col in list(solo_en_original)[:5]:
        print(f"   • {col}")
    if len(solo_en_original) > 5:
        print(f"   ... y {len(solo_en_original) - 5} más")

if solo_en_cleaned:
    print(f"\n⚠️  Columnas válidas SOLO en Cleaned ({len(solo_en_cleaned)}):")
    for col in list(solo_en_cleaned)[:5]:
        print(f"   • {col}")
    if len(solo_en_cleaned) > 5:
        print(f"   ... y {len(solo_en_cleaned) - 5} más")

# Mostrar columnas problemáticas si las hay
if cols_prob_orig:
    print(f"\n❌ Columnas problemáticas en Original ({len(cols_prob_orig)}):")
    for item in cols_prob_orig[:5]:
        print(f"   • {item['columna']}: {item['pct_validos']:.1f}% válidos, "
              f"{item['valores_invalidos']} valores inválidos")
    if len(cols_prob_orig) > 5:
        print(f"   ... y {len(cols_prob_orig) - 5} más")

if cols_prob_clean:
    print(f"\n❌ Columnas problemáticas en Cleaned ({len(cols_prob_clean)}):")
    for item in cols_prob_clean[:5]:
        print(f"   • {item['columna']}: {item['pct_validos']:.1f}% válidos, "
              f"{item['valores_invalidos']} valores inválidos")
    if len(cols_prob_clean) > 5:
        print(f"   ... y {len(cols_prob_clean) - 5} más")

# Reemplazar las variables originales con las versiones limpias
df_original = df_original_clean
df_cleaned = df_cleaned_clean
columnas_numericas = columnas_numericas_validas

print(f"\n" + "="*80)
print(f"✅ LIMPIEZA COMPLETADA")
print(f"   De ahora en adelante se usarán {len(columnas_numericas)} columnas numéricas validadas")
print(f"   Los datasets han sido actualizados con valores convertidos a numérico")
print("="*80)

In [ ]:
# ==============================================================================
# VERIFICACIÓN POST-LIMPIEZA
# ==============================================================================
print("="*80)
print("🔍 VERIFICACIÓN DE TIPOS DE DATOS POST-LIMPIEZA")
print("="*80)

print(f"\n📊 Verificando {len(columnas_numericas)} columnas numéricas validadas...")
print("-"*80)

# Verificar tipos en Original
print("\n✅ DATASET ORIGINAL - Tipos de datos:")
tipos_correctos_orig = 0
for col in columnas_numericas[:10]:  # Mostrar primeras 10
    tipo = df_original[col].dtype
    es_numerico = np.issubdtype(tipo, np.number)
    status = "✅" if es_numerico else "❌"
    if es_numerico:
        tipos_correctos_orig += 1
    print(f"  {status} {col:<40} {str(tipo):<15}")

if len(columnas_numericas) > 10:
    print(f"  ... y {len(columnas_numericas) - 10} columnas más")

# Verificar tipos en Cleaned
print("\n✅ DATASET CLEANED - Tipos de datos:")
tipos_correctos_clean = 0
for col in columnas_numericas[:10]:  # Mostrar primeras 10
    tipo = df_cleaned[col].dtype
    es_numerico = np.issubdtype(tipo, np.number)
    status = "✅" if es_numerico else "❌"
    if es_numerico:
        tipos_correctos_clean += 1
    print(f"  {status} {col:<40} {str(tipo):<15}")

if len(columnas_numericas) > 10:
    print(f"  ... y {len(columnas_numericas) - 10} columnas más")

print("\n" + "="*80)
print(f"✅ Verificación completada:")
print(f"   • Todas las {len(columnas_numericas)} columnas son numéricas")
print(f"   • Listas para análisis estadístico y comparativo")
print("="*80)

---
## 3. Análisis de Valores Nulos y Duplicados
---

In [ ]:
# Función para analizar valores nulos
def analizar_valores_nulos(df, nombre):
    print(f"\n{'='*80}")
    print(f"ANÁLISIS DE VALORES NULOS - {nombre}")
    print(f"{'='*80}")
    
    nulos = df.isnull().sum()
    porcentaje_nulos = (nulos / len(df)) * 100
    
    tabla_nulos = pd.DataFrame({
        'Valores Nulos': nulos,
        'Porcentaje (%)': porcentaje_nulos
    })
    
    tabla_nulos = tabla_nulos[tabla_nulos['Valores Nulos'] > 0].sort_values(
        by='Valores Nulos', ascending=False
    )
    
    if len(tabla_nulos) > 0:
        print(f"\n⚠️  Se encontraron {len(tabla_nulos)} columnas con valores nulos:\n")
        display(tabla_nulos)
    else:
        print("\n✅ No se encontraron valores nulos")
    
    return tabla_nulos

# Analizar ambos datasets
nulos_original = analizar_valores_nulos(df_original, "DATASET ORIGINAL")
nulos_cleaned = analizar_valores_nulos(df_cleaned, "DATASET CLEANED")

In [ ]:
# Análisis de duplicados
print("="*80)
print("ANÁLISIS DE DUPLICADOS")
print("="*80)

duplicados_original = df_original.duplicated().sum()
duplicados_cleaned = df_cleaned.duplicated().sum()

print(f"\n📊 Dataset Original: {duplicados_original} filas duplicadas")
print(f"📊 Dataset Cleaned:  {duplicados_cleaned} filas duplicadas")

if duplicados_original > 0 or duplicados_cleaned > 0:
    print(f"\n⚠️  Diferencia: {abs(duplicados_cleaned - duplicados_original)} duplicados")
else:
    print("\n✅ Ningún dataset tiene duplicados")

---
## 4. Comparación de Distribución de Clases
---

In [ ]:
# Distribución de clases
print("="*80)
print("DISTRIBUCIÓN DE CLASES DE EMOCIÓN")
print("="*80)

dist_original = df_original['Class'].value_counts().sort_index()
dist_cleaned = df_cleaned['Class'].value_counts().sort_index()

comparacion_clases = pd.DataFrame({
    'Original': dist_original,
    'Cleaned': dist_cleaned,
    'Diferencia': dist_cleaned - dist_original,
    'Cambio (%)': ((dist_cleaned - dist_original) / dist_original * 100).round(2)
})

print("\n📊 Comparación de distribución de clases:\n")
display(comparacion_clases)

# Visualización
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Dataset Original
axes[0].bar(dist_original.index, dist_original.values, color='steelblue', alpha=0.7, edgecolor='black')
axes[0].set_title('Distribución de Clases - Dataset Original', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Clase de Emoción', fontsize=12)
axes[0].set_ylabel('Frecuencia', fontsize=12)
axes[0].grid(axis='y', alpha=0.3)

for i, v in enumerate(dist_original.values):
    axes[0].text(i, v + 2, str(v), ha='center', va='bottom', fontweight='bold')

# Dataset Cleaned
axes[1].bar(dist_cleaned.index, dist_cleaned.values, color='seagreen', alpha=0.7, edgecolor='black')
axes[1].set_title('Distribución de Clases - Dataset Cleaned', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Clase de Emoción', fontsize=12)
axes[1].set_ylabel('Frecuencia', fontsize=12)
axes[1].grid(axis='y', alpha=0.3)

for i, v in enumerate(dist_cleaned.values):
    axes[1].text(i, v + 2, str(v), ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# ==============================================================================
# Gráfico de Comparación Lado a Lado (Con Manejo de Clases Diferentes)
# ==============================================================================

# Obtener todas las clases únicas de ambos datasets
all_classes = sorted(set(dist_original.index) | set(dist_cleaned.index))

# Crear DataFrames con todas las clases, rellenando con 0 las faltantes
dist_original_aligned = pd.Series(
    [dist_original.get(cls, 0) for cls in all_classes], 
    index=all_classes
)
dist_cleaned_aligned = pd.Series(
    [dist_cleaned.get(cls, 0) for cls in all_classes], 
    index=all_classes
)

# Crear el gráfico
fig, ax = plt.subplots(figsize=(14, 7))

x = np.arange(len(all_classes))
width = 0.35

bars1 = ax.bar(x - width/2, dist_original_aligned.values, width, label='Original',
               color='steelblue', alpha=0.8, edgecolor='black')
bars2 = ax.bar(x + width/2, dist_cleaned_aligned.values, width, label='Cleaned',
               color='seagreen', alpha=0.8, edgecolor='black')

ax.set_xlabel('Clase de Emoción', fontsize=12, fontweight='bold')
ax.set_ylabel('Frecuencia', fontsize=12, fontweight='bold')
ax.set_title('Comparación de Distribución de Clases: Original vs Cleaned',
             fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(all_classes, rotation=45, ha='right')
ax.legend(fontsize=11)
ax.grid(axis='y', alpha=0.3)

# Agregar valores en las barras
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        if height > 0:  # Solo mostrar si hay valores
            ax.text(bar.get_x() + bar.get_width()/2., height,
                    f'{int(height)}',
                    ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

# Mostrar información sobre las diferencias de clases
print("\n" + "="*80)
print("📊 ANÁLISIS DE CLASES")
print("="*80)

print(f"\n🔍 Total de clases únicas: {len(all_classes)}")
print(f"   Clases: {', '.join(all_classes)}")

# Clases solo en Original
only_in_original = set(dist_original.index) - set(dist_cleaned.index)
if only_in_original:
    print(f"\n⚠️  Clases solo en Original: {', '.join(only_in_original)}")
    for cls in only_in_original:
        print(f"   • {cls}: {dist_original[cls]} muestras")

# Clases solo en Cleaned
only_in_cleaned = set(dist_cleaned.index) - set(dist_original.index)
if only_in_cleaned:
    print(f"\n⚠️  Clases solo en Cleaned: {', '.join(only_in_cleaned)}")
    for cls in only_in_cleaned:
        print(f"   • {cls}: {dist_cleaned[cls]} muestras")

# Clases comunes
common_classes = set(dist_original.index) & set(dist_cleaned.index)
if common_classes:
    print(f"\n✅ Clases comunes: {', '.join(sorted(common_classes))}")
    print("\n   Comparación:")
    print(f"   {'Clase':<15} {'Original':<12} {'Cleaned':<12} {'Diferencia':<12}")
    print("   " + "-"*50)
    for cls in sorted(common_classes):
        diff = dist_cleaned[cls] - dist_original[cls]
        print(f"   {cls:<15} {dist_original[cls]:<12} {dist_cleaned[cls]:<12} {diff:+<12}")

print("\n" + "="*80)

---
## 5. Estadísticas Descriptivas Comparativas
---

In [ ]:
print("="*80)
print("ESTADÍSTICAS DESCRIPTIVAS - DATASET ORIGINAL")
print("="*80)
display(df_original[columnas_numericas].describe())

In [ ]:
print("="*80)
print("ESTADÍSTICAS DESCRIPTIVAS - DATASET CLEANED")
print("="*80)
display(df_cleaned[columnas_numericas].describe())

---
## 6. Análisis de Outliers Comparativo
---

In [ ]:
# ==============================================================================
# ANÁLISIS DE OUTLIERS COMPARATIVO (Método IQR)
# ==============================================================================

# Función mejorada para detectar outliers usando IQR con validación
def detectar_outliers_iqr(df, columnas):
    """
    Detecta outliers usando el método IQR (Interquartile Range).
    Solo procesa columnas que sean realmente numéricas.
    """
    outliers_dict = {}
    columnas_procesadas = []
    columnas_omitidas = []
    
    for col in columnas:
        try:
            # Verificar que la columna sea numérica y no tenga solo strings
            if pd.api.types.is_numeric_dtype(df[col]):
                # Convertir a numeric por si acaso hay valores mixtos
                serie_numerica = pd.to_numeric(df[col], errors='coerce')
                
                # Eliminar NaN para el cálculo
                serie_limpia = serie_numerica.dropna()
                
                if len(serie_limpia) > 0:
                    Q1 = serie_limpia.quantile(0.25)
                    Q3 = serie_limpia.quantile(0.75)
                    IQR = Q3 - Q1
                    
                    lower_bound = Q1 - 1.5 * IQR
                    upper_bound = Q3 + 1.5 * IQR
                    
                    # Contar outliers
                    outliers = serie_limpia[(serie_limpia < lower_bound) | (serie_limpia > upper_bound)]
                    outliers_dict[col] = len(outliers)
                    columnas_procesadas.append(col)
                else:
                    outliers_dict[col] = 0
                    columnas_omitidas.append((col, "sin valores válidos"))
            else:
                outliers_dict[col] = 0
                columnas_omitidas.append((col, "no numérica"))
                
        except Exception as e:
            print(f"⚠️  Error procesando columna '{col}': {str(e)}")
            outliers_dict[col] = 0
            columnas_omitidas.append((col, f"error: {str(e)[:50]}"))
    
    return outliers_dict, columnas_procesadas, columnas_omitidas

print("="*80)
print("ANÁLISIS DE OUTLIERS (Método IQR)")
print("="*80)

# Detectar outliers en ambos datasets
print("\n🔍 Procesando dataset Original...")
outliers_original, cols_proc_orig, cols_omit_orig = detectar_outliers_iqr(df_original, columnas_numericas)

print("🔍 Procesando dataset Cleaned...")
outliers_cleaned, cols_proc_clean, cols_omit_clean = detectar_outliers_iqr(df_cleaned, columnas_numericas)

# Mostrar columnas omitidas si las hay
if cols_omit_orig:
    print(f"\n⚠️  Columnas omitidas en Original ({len(cols_omit_orig)}):")
    for col, razon in cols_omit_orig[:5]:  # Mostrar solo las primeras 5
        print(f"   • {col}: {razon}")
    if len(cols_omit_orig) > 5:
        print(f"   ... y {len(cols_omit_orig) - 5} más")

if cols_omit_clean:
    print(f"\n⚠️  Columnas omitidas en Cleaned ({len(cols_omit_clean)}):")
    for col, razon in cols_omit_clean[:5]:
        print(f"   • {col}: {razon}")
    if len(cols_omit_clean) > 5:
        print(f"   ... y {len(cols_omit_clean) - 5} más")

# Obtener solo las columnas que se procesaron exitosamente en ambos datasets
columnas_validas = list(set(cols_proc_orig) & set(cols_proc_clean))

print(f"\n✅ Columnas procesadas exitosamente: {len(columnas_validas)} de {len(columnas_numericas)}")

# Crear tabla comparativa solo con columnas válidas
if columnas_validas:
    comparacion_outliers = pd.DataFrame({
        'Original': {col: outliers_original[col] for col in columnas_validas},
        'Cleaned': {col: outliers_cleaned[col] for col in columnas_validas},
        'Reducción': {col: outliers_original[col] - outliers_cleaned[col] for col in columnas_validas},
        'Reducción (%)': {
            col: ((outliers_original[col] - outliers_cleaned[col]) / max(outliers_original[col], 1) * 100)
            for col in columnas_validas
        }
    })
    
    # Filtrar solo las que tienen outliers en Original
    comparacion_outliers = comparacion_outliers[comparacion_outliers['Original'] > 0].sort_values(
        by='Reducción', ascending=False
    )
    
    print("\n" + "="*80)
    print("COMPARACIÓN DE OUTLIERS (Método IQR)")
    print("="*80)
    print(f"\n📊 Total de outliers en Original: {sum(outliers_original.values()):,}")
    print(f"📊 Total de outliers en Cleaned:  {sum(outliers_cleaned.values()):,}")
    print(f"📊 Reducción total:                {sum(outliers_original.values()) - sum(outliers_cleaned.values()):,} outliers")
    
    reduccion_pct = 0
    if sum(outliers_original.values()) > 0:
        reduccion_pct = ((sum(outliers_original.values()) - sum(outliers_cleaned.values())) / 
                         sum(outliers_original.values()) * 100)
    print(f"📊 Reducción porcentual:           {reduccion_pct:.2f}%")
    
    if len(comparacion_outliers) > 0:
        print(f"\n📈 Top 10 variables con más outliers reducidos:\n")
        display(comparacion_outliers.head(10))
    else:
        print("\n✅ No se detectaron outliers en el dataset Original")
else:
    print("\n❌ No se pudieron procesar columnas numéricas para análisis de outliers")

print("\n" + "="*80)

In [ ]:
# Visualización de outliers
if len(comparacion_outliers) > 0:
    top_features = comparacion_outliers.head(10).index.tolist()
    
    fig, ax = plt.subplots(figsize=(14, 8))
    
    x = np.arange(len(top_features))
    width = 0.35
    
    bars1 = ax.bar(x - width/2, [outliers_original[f] for f in top_features], 
                   width, label='Original', color='crimson', alpha=0.7, edgecolor='black')
    bars2 = ax.bar(x + width/2, [outliers_cleaned[f] for f in top_features], 
                   width, label='Cleaned', color='forestgreen', alpha=0.7, edgecolor='black')
    
    ax.set_xlabel('Features', fontsize=12, fontweight='bold')
    ax.set_ylabel('Cantidad de Outliers', fontsize=12, fontweight='bold')
    ax.set_title('Top 10 Features con Más Outliers: Original vs Cleaned', 
                 fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels([f.replace('_', ' ') for f in top_features], rotation=45, ha='right')
    ax.legend(fontsize=11)
    ax.grid(axis='y', alpha=0.3)
    
    # Agregar valores
    for bars in [bars1, bars2]:
        for bar in bars:
            height = bar.get_height()
            if height > 0:
                ax.text(bar.get_x() + bar.get_width()/2., height,
                        f'{int(height)}',
                        ha='center', va='bottom', fontsize=9)
    
    plt.tight_layout()
    plt.show()

---
## 7. Análisis Específico de Diferencias Celda por Celda
---

Como los datasets tienen diferente número de filas, vamos a:
1. Encontrar filas comunes basadas en características similares
2. Analizar las diferencias en los valores numéricos

In [ ]:
# ==============================================================================
# ANÁLISIS DE DIFERENCIAS CELDA POR CELDA (Datasets Alineados)
# ==============================================================================

n_compare = min(len(df_original), len(df_cleaned))

print("="*80)
print(f"ANÁLISIS DE DIFERENCIAS EN LAS PRIMERAS {n_compare} FILAS")
print("="*80)

# Comparar valores para las primeras n filas
diferencias_dict = {}
columnas_procesadas = []
columnas_con_error = []

for col in columnas_numericas:
    try:
        # Obtener valores y convertir a numérico explícitamente
        valores_original = pd.to_numeric(df_original[col].iloc[:n_compare], errors='coerce').values
        valores_cleaned = pd.to_numeric(df_cleaned[col].iloc[:n_compare], errors='coerce').values
        
        # Verificar que tengamos suficientes valores válidos
        valores_validos_orig = ~np.isnan(valores_original)
        valores_validos_clean = ~np.isnan(valores_cleaned)
        valores_validos = valores_validos_orig & valores_validos_clean
        
        if np.sum(valores_validos) < n_compare * 0.5:  # Si menos del 50% son válidos, omitir
            columnas_con_error.append((col, f"Pocos valores válidos: {np.sum(valores_validos)}/{n_compare}"))
            continue
        
        # Calcular diferencias solo en valores válidos
        diff = np.abs(valores_original - valores_cleaned)
        diff_validos = diff[valores_validos]
        
        # Contar cambios (con una pequeña tolerancia para errores de punto flotante)
        tolerancia = 1e-10
        cambios = np.sum(diff_validos > tolerancia)
        
        diferencias_dict[col] = {
            'Cambios_Totales': cambios,
            'Cambios_%': (cambios / np.sum(valores_validos)) * 100,
            'Diff_Media': np.mean(diff_validos[diff_validos > tolerancia]) if cambios > 0 else 0,
            'Diff_Max': np.max(diff_validos),
            'Diff_Min': np.min(diff_validos[diff_validos > tolerancia]) if cambios > 0 else 0,
            'N_Valores_Validos': np.sum(valores_validos)
        }
        
        columnas_procesadas.append(col)
        
    except Exception as e:
        columnas_con_error.append((col, str(e)[:50]))
        continue

# Mostrar columnas con errores si las hay
if columnas_con_error:
    print(f"\n⚠️  Columnas omitidas del análisis ({len(columnas_con_error)}):")
    for col, error in columnas_con_error[:10]:  # Mostrar primeras 10
        print(f"   • {col}: {error}")
    if len(columnas_con_error) > 10:
        print(f"   ... y {len(columnas_con_error) - 10} más")

print(f"\n✅ Columnas procesadas exitosamente: {len(columnas_procesadas)}")

# Crear DataFrame de diferencias
if len(diferencias_dict) > 0:
    df_diferencias = pd.DataFrame(diferencias_dict).T
    df_diferencias = df_diferencias[df_diferencias['Cambios_Totales'] > 0].sort_values(
        by='Cambios_Totales', ascending=False
    )
    
    print(f"\n📊 Columnas que presentan cambios: {len(df_diferencias)} de {len(columnas_procesadas)}")
    
    if len(df_diferencias) > 0:
        print(f"\nTop 15 columnas con más cambios:\n")
        display(df_diferencias.head(15))
        
        # Calcular estadísticas totales
        total_celdas = n_compare * len(columnas_procesadas)
        celdas_modificadas = df_diferencias['Cambios_Totales'].sum()
        
        print(f"\n📊 RESUMEN DE CAMBIOS:")
        print("-"*80)
        print(f"   • Total de celdas analizadas:     {total_celdas:,}")
        print(f"   • Celdas modificadas:             {int(celdas_modificadas):,}")
        print(f"   • Porcentaje modificado:          {(celdas_modificadas/total_celdas)*100:.2f}%")
        print(f"   • Columnas con cambios:           {len(df_diferencias)}")
        print(f"   • Columnas sin cambios:           {len(columnas_procesadas) - len(df_diferencias)}")
    else:
        print("\n✅ No se detectaron cambios significativos entre los datasets")
else:
    print("\n❌ No se pudieron procesar columnas para análisis de diferencias")

print("\n" + "="*80)

In [ ]:
# ==============================================================================
# ANÁLISIS DETALLADO DE COLUMNA ESPECÍFICA (Podemos Indicar la Columna Deseada)
# ==============================================================================

if len(df_diferencias) > 0:
    # Seleccionar la columna con más cambios
    columna_mas_cambios = df_diferencias.index[0]
    
    print("="*80)
    print(f"ANÁLISIS DETALLADO: {columna_mas_cambios}")
    print("="*80)
    
    # Obtener valores
    vals_orig = pd.to_numeric(df_original[columna_mas_cambios].iloc[:n_compare], errors='coerce')
    vals_clean = pd.to_numeric(df_cleaned[columna_mas_cambios].iloc[:n_compare], errors='coerce')
    
    # Calcular diferencias
    diferencias = vals_clean - vals_orig
    diferencias_abs = np.abs(diferencias)
    
    # Encontrar índices de cambios
    indices_cambios = np.where(diferencias_abs > 1e-10)[0]
    
    print(f"\n📊 Estadísticas de la columna:")
    print("-"*80)
    print(f"   • Total de cambios:           {len(indices_cambios)}")
    print(f"   • Porcentaje de cambios:      {(len(indices_cambios)/n_compare)*100:.2f}%")
    print(f"   • Diferencia promedio:        {np.mean(diferencias_abs[indices_cambios]):.6f}")
    print(f"   • Diferencia máxima:          {np.max(diferencias_abs):.6f}")
    print(f"   • Diferencia mínima (>0):     {np.min(diferencias_abs[indices_cambios]):.6f}")
    
    # Mostrar algunos ejemplos de cambios
    print(f"\n📋 Ejemplos de cambios (primeros 10):")
    print("-"*80)
    print(f"   {'Índice':<8} {'Original':<15} {'Cleaned':<15} {'Diferencia':<15}")
    print("   " + "-"*55)
    
    for idx in indices_cambios[:10]:
        orig_val = vals_orig.iloc[idx]
        clean_val = vals_clean.iloc[idx]
        diff_val = diferencias.iloc[idx]
        print(f"   {idx:<8} {orig_val:<15.6f} {clean_val:<15.6f} {diff_val:<+15.6f}")
    
    if len(indices_cambios) > 10:
        print(f"   ... y {len(indices_cambios) - 10} cambios más")
    
    # Visualización de diferencias
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Histograma de diferencias
    axes[0].hist(diferencias[indices_cambios], bins=30, color='coral', 
                 alpha=0.7, edgecolor='black')
    axes[0].axvline(x=0, color='red', linestyle='--', linewidth=2, label='Sin cambio')
    axes[0].set_title(f'Distribución de Diferencias\n{columna_mas_cambios}', 
                      fontsize=12, fontweight='bold')
    axes[0].set_xlabel('Diferencia (Cleaned - Original)', fontsize=10)
    axes[0].set_ylabel('Frecuencia', fontsize=10)
    axes[0].legend()
    axes[0].grid(alpha=0.3)
    
    # Scatter plot: Original vs Cleaned
    axes[1].scatter(vals_orig, vals_clean, alpha=0.5, s=30)
    
    # Línea de igualdad (y = x)
    min_val = min(vals_orig.min(), vals_clean.min())
    max_val = max(vals_orig.max(), vals_clean.max())
    axes[1].plot([min_val, max_val], [min_val, max_val], 
                 'r--', linewidth=2, label='y = x (Sin cambios)')
    
    axes[1].set_title(f'Comparación: Original vs Cleaned\n{columna_mas_cambios}', 
                      fontsize=12, fontweight='bold')
    axes[1].set_xlabel('Valores Original', fontsize=10)
    axes[1].set_ylabel('Valores Cleaned', fontsize=10)
    axes[1].legend()
    axes[1].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\n" + "="*80)

---
## 8. Comparación de Distribuciones de Features Clave
---

In [ ]:
# Seleccionar features clave para comparar distribuciones
features_clave = [
    '_RMSenergy_Mean',
    '_Tempo_Mean',
    '_Roughness_Mean',
    '_Brightness_Mean',
    '_Spectralcentroid_Mean',
    '_Zero-crossingrate_Mean'
]

# Verificar que existen
features_clave = [f for f in features_clave if f in columnas_numericas]

fig, axes = plt.subplots(3, 2, figsize=(16, 14))
axes = axes.flatten()

for idx, feature in enumerate(features_clave):
    ax = axes[idx]
    
    # Histogramas superpuestos
    ax.hist(df_original[feature], bins=30, alpha=0.5, label='Original', 
            color='steelblue', edgecolor='black')
    ax.hist(df_cleaned[feature], bins=30, alpha=0.5, label='Cleaned', 
            color='seagreen', edgecolor='black')
    
    ax.set_title(f'Distribución: {feature.replace("_", " ")}', 
                 fontsize=11, fontweight='bold')
    ax.set_xlabel('Valor', fontsize=10)
    ax.set_ylabel('Frecuencia', fontsize=10)
    ax.legend()
    ax.grid(alpha=0.3)

plt.suptitle('Comparación de Distribuciones de Features Clave', 
             fontsize=14, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

---
## 9. Test Estadísticos de Similitud
---

In [ ]:
# Realizar test de Kolmogorov-Smirnov para comparar distribuciones
print("="*80)
print("TEST DE KOLMOGOROV-SMIRNOV")
print("Compara si dos muestras provienen de la misma distribución")
print("H0: Las distribuciones son iguales | p-value > 0.05 → No rechazamos H0")
print("="*80)

ks_results = {}

for col in columnas_numericas:
    # Usar solo las primeras n_compare filas
    stat, pvalue = stats.ks_2samp(
        df_original[col].iloc[:n_compare],
        df_cleaned[col].iloc[:n_compare]
    )
    
    ks_results[col] = {
        'KS_Statistic': stat,
        'p_value': pvalue,
        'Distribuciones_Similares': 'Sí' if pvalue > 0.05 else 'No'
    }

df_ks = pd.DataFrame(ks_results).T.sort_values(by='p_value')

print(f"\nColumnas con distribuciones SIGNIFICATIVAMENTE DIFERENTES (p < 0.05):\n")
diferentes = df_ks[df_ks['p_value'] < 0.05]
if len(diferentes) > 0:
    display(diferentes.head(15))
    print(f"\n⚠️  {len(diferentes)} columnas tienen distribuciones significativamente diferentes")
else:
    print("✅ Todas las distribuciones son estadísticamente similares")

print(f"\n\nColumnas con distribuciones SIMILARES (p >= 0.05):\n")
similares = df_ks[df_ks['p_value'] >= 0.05]
if len(similares) > 0:
    display(similares.head(15))
    print(f"\n✅ {len(similares)} columnas mantienen distribuciones similares")

---
## 10. Métricas Globales de Comparación
---

In [ ]:
# Calcular métricas globales
print("="*80)
print("MÉTRICAS GLOBALES DE COMPARACIÓN")
print("="*80)

metricas = {}

# 1. Cambio en tamaño del dataset
metricas['Filas_Original'] = len(df_original)
metricas['Filas_Cleaned'] = len(df_cleaned)
metricas['Diferencia_Filas'] = len(df_cleaned) - len(df_original)
metricas['Cambio_Porcentual_Filas'] = ((len(df_cleaned) - len(df_original)) / len(df_original)) * 100

# 2. Cambio en valores nulos
metricas['Nulos_Original'] = df_original.isnull().sum().sum()
metricas['Nulos_Cleaned'] = df_cleaned.isnull().sum().sum()
metricas['Reduccion_Nulos'] = metricas['Nulos_Original'] - metricas['Nulos_Cleaned']

# 3. Cambio en outliers
metricas['Outliers_Original'] = sum(outliers_original.values())
metricas['Outliers_Cleaned'] = sum(outliers_cleaned.values())
metricas['Reduccion_Outliers'] = metricas['Outliers_Original'] - metricas['Outliers_Cleaned']

# 4. Cambio en duplicados
metricas['Duplicados_Original'] = duplicados_original
metricas['Duplicados_Cleaned'] = duplicados_cleaned

# 5. Porcentaje de valores modificados
if len(df_diferencias) > 0:
    metricas['Columnas_Modificadas'] = len(df_diferencias)
    metricas['Porcentaje_Columnas_Modificadas'] = (len(df_diferencias) / len(columnas_numericas)) * 100
    metricas['Total_Celdas_Modificadas'] = df_diferencias['Cambios_Totales'].sum()
else:
    metricas['Columnas_Modificadas'] = 0
    metricas['Porcentaje_Columnas_Modificadas'] = 0
    metricas['Total_Celdas_Modificadas'] = 0

# 6. Distribuciones similares
metricas['Distribuciones_Similares'] = len(similares)
metricas['Distribuciones_Diferentes'] = len(diferentes)
metricas['Porcentaje_Similares'] = (len(similares) / len(columnas_numericas)) * 100

# Mostrar métricas
df_metricas = pd.DataFrame(list(metricas.items()), columns=['Métrica', 'Valor'])
display(df_metricas)

In [ ]:
# 1. FORZAR CIERRE DE FIGURAS ANTERIORES
plt.close('all')

# 2. CREAR LA FIGURA Y LOS EJES 
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Gráfico 1: Cambio en tamaño
axes[0, 0].bar(['Original', 'Cleaned'],
               [metricas['Filas_Original'], metricas['Filas_Cleaned']],
               color=['steelblue', 'seagreen'], alpha=0.7, edgecolor='black')
axes[0, 0].set_title('Tamaño del Dataset', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('Número de Filas', fontsize=11)
axes[0, 0].grid(axis='y', alpha=0.3)
for i, v in enumerate([metricas['Filas_Original'], metricas['Filas_Cleaned']]):
    axes[0, 0].text(i, v + 5, str(v), ha='center', va='bottom', fontweight='bold')

# Gráfico 2: Cambio en outliers
axes[0, 1].bar(['Original', 'Cleaned'],
               [metricas['Outliers_Original'], metricas['Outliers_Cleaned']],
               color=['crimson', 'forestgreen'], alpha=0.7, edgecolor='black')
axes[0, 1].set_title('Total de Outliers Detectados', fontsize=12, fontweight='bold')
axes[0, 1].set_ylabel('Cantidad de Outliers', fontsize=11)
axes[0, 1].grid(axis='y', alpha=0.3)
for i, v in enumerate([metricas['Outliers_Original'], metricas['Outliers_Cleaned']]):
    axes[0, 1].text(i, v + 10, str(v), ha='center', va='bottom', fontweight='bold')

# Gráfico 3: Distribuciones similares vs diferentes
axes[1, 0].bar(['Similares', 'Diferentes'],
               [metricas['Distribuciones_Similares'], metricas['Distribuciones_Diferentes']],
               color=['mediumseagreen', 'coral'], alpha=0.7, edgecolor='black')
axes[1, 0].set_title('Comparación de Distribuciones (Test KS)', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('Número de Columnas', fontsize=11)
axes[1, 0].grid(axis='y', alpha=0.3)
for i, v in enumerate([metricas['Distribuciones_Similares'], metricas['Distribuciones_Diferentes']]):
    axes[1, 0].text(i, v + 0.001, str(v), ha='center', va='bottom', fontweight='bold') # Ajuste pequeño en 'y'

# Gráfico 4: Resumen de cambios
categorias = ['Valores\nNulos', 'Outliers', 'Columnas\nModificadas']
valores = [
    metricas['Reduccion_Nulos'],
    metricas['Reduccion_Outliers'],
    metricas['Columnas_Modificadas']
]
colores = ['dodgerblue', 'orange', 'mediumpurple']

axes[1, 1].bar(categorias, valores, color=colores, alpha=0.7, edgecolor='black')
axes[1, 1].set_title('Resumen de Cambios Aplicados', fontsize=12, fontweight='bold')
axes[1, 1].set_ylabel('Cantidad', fontsize=11)
axes[1, 1].grid(axis='y', alpha=0.3)
for i, v in enumerate(valores):
    # Ajuste para que el texto aparezca correctamente si los valores son negativos
    offset = max(valores) * 0.05
    y_pos = v + offset if v >= 0 else v - offset
    va = 'bottom' if v >= 0 else 'top'
    axes[1, 1].text(i, y_pos, str(v), ha='center', va=va, fontweight='bold')


# 3. TÍTULO GENERAL Y AJUSTE AUTOMÁTICO
fig.suptitle('Métricas Globales de Comparación: Original vs Cleaned',
             fontsize=14, fontweight='bold')

# Usamos el ajuste automático que es más robusto después de un reinicio
fig.tight_layout(rect=[0, 0, 1, 0.96])

plt.show()

---
## 11. Preparación para Alineación de Datasets
---

Para hacer los datasets completamente comparables, necesitamos que tengan el mismo número de filas.
Vamos a crear una versión alineada del dataset cleaned que coincida con el original.

In [ ]:
print("="*80)
print("ESTRATEGIA DE ALINEACIÓN DE DATASETS")
print("="*80)

print(f"\nDataset Original: {len(df_original)} filas")
print(f"Dataset Cleaned:  {len(df_cleaned)} filas")
print(f"Diferencia:       {len(df_cleaned) - len(df_original)} filas extra en Cleaned")

print("\n" + "="*80)
print("OPCIONES DE ALINEACIÓN")
print("="*80)

print("""
Para hacer los datasets comparables, podemos:

1. **Usar las primeras N filas comunes**: Tomar solo las primeras 400 filas de ambos
   ✅ Ventaja: Mantiene el orden original
   ⚠️  Desventaja: Perdemos información de filas adicionales

2. **Usar muestreo aleatorio**: Reducir Cleaned a 400 filas aleatorias
   ✅ Ventaja: Más representativo de todo el dataset
   ⚠️  Desventaja: Perdemos reproducibilidad sin semilla fija

3. **Mantener Cleaned completo para análisis**: Usar todo Cleaned para métricas
   ✅ Ventaja: No perdemos información
   ⚠️  Desventaja: Comparaciones directas más complejas

Implementaremos la OPCIÓN 1 para el análisis comparativo directo.
""")

# Crear versiones alineadas
n_rows_common = len(df_original)

df_cleaned_aligned = df_cleaned.head(n_rows_common).copy()

print(f"\n✅ Dataset Cleaned Alineado creado: {len(df_cleaned_aligned)} filas")
print(f"✅ Ahora Original y Cleaned Alineado tienen el mismo tamaño: {len(df_original)} filas")

---
## 12. Comparación Detallada con Datasets Alineados
---

In [ ]:
# Análisis celda por celda con datasets alineados
print("="*80)
print("ANÁLISIS DETALLADO CELDA POR CELDA (Datasets Alineados)")
print("="*80)

diferencias_detalladas = {}
total_celdas = len(df_original) * len(columnas_numericas)
celdas_modificadas = 0

for col in columnas_numericas:
    valores_original = df_original[col].values
    valores_cleaned = df_cleaned_aligned[col].values
    
    # Calcular diferencias
    diff = np.abs(valores_original - valores_cleaned)
    n_cambios = np.sum(diff > 1e-10)  # Tolerancia numérica
    celdas_modificadas += n_cambios
    
    if n_cambios > 0:
        diferencias_detalladas[col] = {
            'N_Cambios': n_cambios,
            'Cambios_%': (n_cambios / len(df_original)) * 100,
            'Diff_Media': np.mean(diff[diff > 1e-10]),
            'Diff_Mediana': np.median(diff[diff > 1e-10]),
            'Diff_Max': np.max(diff),
            'Diff_Std': np.std(diff[diff > 1e-10])
        }

df_diffs_detalladas = pd.DataFrame(diferencias_detalladas).T.sort_values(
    by='N_Cambios', ascending=False
)

print(f"\n📊 Total de celdas en el dataset: {total_celdas:,}")
print(f"📊 Celdas modificadas: {celdas_modificadas:,}")
print(f"📊 Porcentaje de celdas modificadas: {(celdas_modificadas/total_celdas)*100:.2f}%")

if len(df_diffs_detalladas) > 0:
    print(f"\nTop 20 columnas con más cambios:\n")
    display(df_diffs_detalladas.head(20))
else:
    print("\n✅ No se detectaron cambios significativos entre los datasets")

In [ ]:
# Visualización de las columnas más modificadas
if len(df_diffs_detalladas) > 0:
    top_changed = df_diffs_detalladas.head(12)
    
    fig, axes = plt.subplots(3, 4, figsize=(18, 12))
    axes = axes.flatten()
    
    for idx, (col, row) in enumerate(top_changed.iterrows()):
        if idx >= 12:
            break
            
        ax = axes[idx]
        
        # Calcular diferencias
        valores_original = df_original[col].values
        valores_cleaned = df_cleaned_aligned[col].values
        diff = valores_cleaned - valores_original
        
        # Histograma de diferencias
        ax.hist(diff, bins=30, color='coral', alpha=0.7, edgecolor='black')
        ax.axvline(x=0, color='red', linestyle='--', linewidth=2, label='Sin cambio')
        ax.set_title(f'{col.replace("_", " ")}\n{int(row["N_Cambios"])} cambios ({row["Cambios_%"]:.1f}%)', 
                     fontsize=9, fontweight='bold')
        ax.set_xlabel('Diferencia (Cleaned - Original)', fontsize=8)
        ax.set_ylabel('Frecuencia', fontsize=8)
        ax.grid(alpha=0.3)
        ax.legend(fontsize=7)
    
    # Ocultar axes vacíos
    for idx in range(len(top_changed), 12):
        axes[idx].axis('off')
    
    plt.suptitle('Distribución de Diferencias en Top 12 Columnas Más Modificadas', 
                 fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

---
## 13. Análisis de Correlación Comparativo
---

In [ ]:
# Calcular matrices de correlación
corr_original = df_original[columnas_numericas].corr()
corr_cleaned = df_cleaned_aligned[columnas_numericas].corr()

# Calcular diferencia en correlaciones
corr_diff = np.abs(corr_cleaned - corr_original)

print("="*80)
print("ANÁLISIS DE CAMBIOS EN CORRELACIONES")
print("="*80)

# Encontrar las correlaciones que más cambiaron
cambios_corr = []
for i in range(len(corr_diff)):
    for j in range(i+1, len(corr_diff)):
        cambios_corr.append({
            'Feature_1': corr_diff.index[i],
            'Feature_2': corr_diff.columns[j],
            'Corr_Original': corr_original.iloc[i, j],
            'Corr_Cleaned': corr_cleaned.iloc[i, j],
            'Diferencia_Abs': corr_diff.iloc[i, j]
        })

df_cambios_corr = pd.DataFrame(cambios_corr).sort_values(
    by='Diferencia_Abs', ascending=False
)

print(f"\nTop 15 pares de features con mayor cambio en correlación:\n")
display(df_cambios_corr.head(15))

print(f"\n📊 Cambio promedio en correlaciones: {corr_diff.values[np.triu_indices_from(corr_diff, k=1)].mean():.4f}")
print(f"📊 Cambio máximo en correlación: {corr_diff.values.max():.4f}")

In [ ]:
# Visualización de diferencias en correlaciones
# Seleccionar un subset de features para visualización más clara
features_subset = columnas_numericas[:15] if len(columnas_numericas) > 15 else columnas_numericas

fig, axes = plt.subplots(1, 3, figsize=(22, 7))

# Matriz de correlación Original
sns.heatmap(corr_original.loc[features_subset, features_subset], 
            annot=False, cmap='coolwarm', center=0, 
            vmin=-1, vmax=1, ax=axes[0], cbar_kws={'label': 'Correlación'})
axes[0].set_title('Matriz de Correlación - Original', fontsize=12, fontweight='bold')

# Matriz de correlación Cleaned
sns.heatmap(corr_cleaned.loc[features_subset, features_subset], 
            annot=False, cmap='coolwarm', center=0, 
            vmin=-1, vmax=1, ax=axes[1], cbar_kws={'label': 'Correlación'})
axes[1].set_title('Matriz de Correlación - Cleaned', fontsize=12, fontweight='bold')

# Diferencia absoluta
sns.heatmap(corr_diff.loc[features_subset, features_subset], 
            annot=False, cmap='Reds', 
            vmin=0, vmax=0.1, ax=axes[2], cbar_kws={'label': 'Diferencia Abs'})
axes[2].set_title('Diferencia Absoluta en Correlaciones', fontsize=12, fontweight='bold')

plt.suptitle('Comparación de Matrices de Correlación (Subset de 15 Features)', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

---
## 14. Exportación de Datasets para DVC
---

In [ ]:
# Ajustar ruta porque el notebook está en notebooks/
output_dir = '../data/processed'  # ← CAMBIO: subir un nivel

# Verificar que existe (por si acaso)
Path(output_dir).mkdir(parents=True, exist_ok=True)

# Dataset original (copia)
df_original.to_csv(f'{output_dir}/turkish_music_emotion_v1_original.csv', index=False)
print(f"✅ Exportado: turkish_music_emotion_v1_original.csv ({len(df_original)} filas)")

# Dataset cleaned alineado (mismo número de filas que original)
df_cleaned_aligned.to_csv(f'{output_dir}/turkish_music_emotion_v2_cleaned_aligned.csv', index=False)
print(f"✅ Exportado: turkish_music_emotion_v2_cleaned_aligned.csv ({len(df_cleaned_aligned)} filas)")

# Dataset cleaned completo (con todas las filas)
df_cleaned.to_csv(f'{output_dir}/turkish_music_emotion_v2_cleaned_full.csv', index=False)
print(f"✅ Exportado: turkish_music_emotion_v2_cleaned_full.csv ({len(df_cleaned)} filas)")

print("\n" + "="*80)
print("ARCHIVOS PREPARADOS PARA DVC")
print("="*80)
print(f"""
✅ Se han creado 3 versiones del dataset en {Path(output_dir).resolve()}:

1. **v1_original**: Dataset original sin modificaciones ({len(df_original)} filas)
   - Para establecer baseline de rendimiento
   - Referencia para comparaciones

2. **v2_cleaned_aligned**: Dataset limpio alineado con original ({len(df_cleaned_aligned)} filas)
   - Mismo tamaño que original para comparaciones directas
   - Incluye todas las transformaciones de limpieza

3. **v2_cleaned_full**: Dataset limpio completo ({len(df_cleaned)} filas)
   - Incluye todas las filas procesadas
   - Para análisis y modelado final

📦 Próximo paso: Versionar con DVC
   Ejecuta en terminal:
   1. cd ..  # Volver a la raíz del proyecto
   2. dvc add data
   3. dvc push
   4. git add data.dvc .gitignore
   5. git commit -m 'feat: add turkish dataset versions v1 and v2'
   6. git push
""")

In [ ]:
# ============================================
# VERSIONAMIENTO AUTOMÁTICO CON DVC + GIT
# ============================================
print("=" * 80)
print("🚀 INICIANDO VERSIONAMIENTO DE DATOS PROCESADOS CON DVC Y GIT")
print("=" * 80)

# 1. Encontrar el directorio correcto del proyecto
current_dir = Path.cwd()
print(f"📁 Directorio inicial: {current_dir}")

# Buscar el directorio con .dvc/ (subir hasta encontrarlo)
if not (current_dir / ".dvc").exists():
    # Si estamos en notebooks/, subir un nivel
    if current_dir.name == "notebooks":
        os.chdir(current_dir.parent)
        current_dir = Path.cwd()
        print(f"✅ Subido desde notebooks/ a: {current_dir}")
    else:
        # Buscar en otras ubicaciones
        potential_dirs = [
            current_dir.parent,  # ← AGREGADO: primero intentar subir un nivel
            current_dir / "MLOps_Team24",
            current_dir.parent / "MLOps_Team24",
        ]
        
        found = False
        for potential_dir in potential_dirs:
            if potential_dir.exists() and (potential_dir / ".dvc").exists():
                os.chdir(potential_dir)
                current_dir = Path.cwd()
                print(f"✅ Cambiado a: {current_dir}")
                found = True
                break
        
        if not found:
            print("\n❌ ERROR: No se encontró el repositorio DVC")
            raise RuntimeError("No estás en un repositorio DVC")
else:
    print(f"✅ Repositorio DVC detectado en: {current_dir}")

# 2. Crear mensaje de commit dinámico
timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
commit_message = f"feat: add turkish dataset versions v1 and v2 - {timestamp}"  # ← Mensaje más específico
print(f"💬 Mensaje de commit: {commit_message}")

# 3. Ejecutar comandos uno por uno
commands = [
    ("dvc add data", "📦 Trackeando cambios en data/ con DVC..."),
    ("dvc push", "☁️  Subiendo a S3..."),
    ("git add data.dvc .gitignore", "📝 Agregando metadatos a Git..."),
    (f'git commit -m "{commit_message}"', "💾 Commiteando cambios..."),
    ("git push", "🚀 Subiendo a GitHub..."),
]

print("\n" + "=" * 80)
success = True
no_changes = False

for cmd, description in commands:
    print(f"\n{description}")
    print(f"   Ejecutando: {cmd}")
    
    result = subprocess.run(
        cmd,
        shell=True,
        capture_output=True,
        text=True,
        cwd=str(current_dir)
    )
    
    # Mostrar output (stdout y stderr combinados)
    output = (result.stdout + result.stderr).strip()
    if output:
        for line in output.split('\n'):
            if line.strip():
                print(f"   {line}")
    
    # Verificar errores
    if result.returncode != 0:
        error_text = result.stderr.lower() + result.stdout.lower()
        
        # Casos NO críticos (normales)
        if "nothing to commit" in error_text or "working tree clean" in error_text:
            print("   ℹ️  No hay cambios nuevos para commitear")
            no_changes = True
            continue  # Continuar con siguiente comando
        elif "everything is up to date" in error_text:
            print("   ℹ️  Todo ya está actualizado")
            continue
        elif "no changes" in error_text:
            print("   ℹ️  No hay cambios en los datos")
            no_changes = True
            continue
        else:
            # Error REAL
            print(f"   ❌ Error crítico: {result.stderr.strip()}")
            success = False
            break

print("\n" + "=" * 80)
if success:
    if no_changes:
        print("✅ PROCESO COMPLETADO - NO HABÍA CAMBIOS")
        print("\n📊 Estado:")
        print("   • Datos ya estaban trackeados con DVC")
        print("   • Respaldo ya estaba en S3")
        print("   • No había cambios que commitear")
        print("   • Todo sincronizado correctamente")
    else:
        print("✅ PROCESO DE VERSIONAMIENTO COMPLETADO EXITOSAMENTE")
        print("\n📊 Resumen:")
        print("   • 3 versiones del dataset turkish añadidas")
        print("   • Datos trackeados con DVC")
        print("   • Respaldo en S3")
        print("   • Metadatos en GitHub")
        print("   • Cambios sincronizados con el equipo")
else:
    print("⚠️  HUBO UN ERROR DURANTE EL PROCESO")
    print("\n💡 Solución manual en terminal:")
    print(f"   cd {current_dir}")
    print("   dvc add data")
    print("   dvc push")
    print("   git add data.dvc .gitignore")
    print(f'   git commit -m "{commit_message}"')
    print("   git push")
    
print("=" * 80)

---
## 15. Reporte Final de Comparación
---

In [ ]:
# Detectar automáticamente dónde estamos
current_dir = Path.cwd()

# Si estamos en notebooks/, subir un nivel. Si ya estamos en la raíz, quedarse ahí
if current_dir.name == 'notebooks':
    reports_dir = current_dir.parent / 'reports'
else:
    # Ya estamos en la raíz del proyecto
    reports_dir = current_dir / 'reports'

# Crear directorio si no existe
reports_dir.mkdir(parents=True, exist_ok=True)

print("="*80)
print("REPORTE FINAL DE COMPARACIÓN")
print("Turkish Music Emotion Dataset: Original vs Cleaned")
print("="*80)

reporte = f"""
╔══════════════════════════════════════════════════════════════════════════════╗
║                         RESUMEN EJECUTIVO                                    ║
╚══════════════════════════════════════════════════════════════════════════════╝

1. TAMAÑO DEL DATASET
   • Original:           {len(df_original):4d} filas × {df_original.shape[1]} columnas
   • Cleaned:            {len(df_cleaned):4d} filas × {df_cleaned.shape[1]} columnas
   • Diferencia:         {len(df_cleaned) - len(df_original):+4d} filas ({((len(df_cleaned) - len(df_original))/len(df_original)*100):+.2f}%)

2. CALIDAD DE DATOS
   • Valores nulos (Original):    {df_original.isnull().sum().sum():6d}
   • Valores nulos (Cleaned):     {df_cleaned.isnull().sum().sum():6d}
   • Reducción de nulos:          {df_original.isnull().sum().sum() - df_cleaned.isnull().sum().sum():6d}
   
   • Duplicados (Original):       {duplicados_original:6d}
   • Duplicados (Cleaned):        {duplicados_cleaned:6d}

3. OUTLIERS
   • Outliers detectados (Original): {sum(outliers_original.values()):6d}
   • Outliers detectados (Cleaned):  {sum(outliers_cleaned.values()):6d}
   • Reducción de outliers:          {sum(outliers_original.values()) - sum(outliers_cleaned.values()):6d} ({((sum(outliers_original.values()) - sum(outliers_cleaned.values()))/max(sum(outliers_original.values()), 1)*100):.1f}%)

4. CAMBIOS EN LOS DATOS (Datasets Alineados)
   • Total de celdas:                {total_celdas:,}
   • Celdas modificadas:             {celdas_modificadas:,}
   • Porcentaje modificado:          {(celdas_modificadas/total_celdas)*100:.2f}%
   • Columnas con cambios:           {len(df_diffs_detalladas)} de {len(columnas_numericas)}

5. DISTRIBUCIONES ESTADÍSTICAS
   • Distribuciones similares (KS test, p>0.05):  {len(similares):3d} ({(len(similares)/len(columnas_numericas)*100):.1f}%)
   • Distribuciones diferentes (KS test, p≤0.05): {len(diferentes):3d} ({(len(diferentes)/len(columnas_numericas)*100):.1f}%)

6. DISTRIBUCIÓN DE CLASES
"""

for clase in dist_original.index:
    orig = dist_original[clase]
    clean = dist_cleaned[clase]
    diff = clean - orig
    reporte += f"   • {clase:8s}: Original={orig:3d}, Cleaned={clean:3d}, Diff={diff:+3d}\n"

reporte += f"""
7. CORRELACIONES
   • Cambio promedio en correlaciones:    {corr_diff.values[np.triu_indices_from(corr_diff, k=1)].mean():.6f}
   • Cambio máximo en correlación:        {corr_diff.values.max():.6f}

╔══════════════════════════════════════════════════════════════════════════════╗
║                         CONCLUSIONES                                         ║
╚══════════════════════════════════════════════════════════════════════════════╝

✅ El proceso de limpieza ha resultado en:

1. MEJORAS EN CALIDAD:
   • {'Eliminación total de valores nulos' if df_cleaned.isnull().sum().sum() == 0 else f'Reducción de {df_original.isnull().sum().sum() - df_cleaned.isnull().sum().sum()} valores nulos'}
   • Reducción de {sum(outliers_original.values()) - sum(outliers_cleaned.values())} outliers
   • Tratamiento de {len(df_diffs_detalladas)} columnas con valores modificados

2. PRESERVACIÓN DE ESTRUCTURA:
   • {len(similares)} de {len(columnas_numericas)} columnas mantienen distribuciones similares
   • Las correlaciones entre features se mantienen estables
   • La distribución de clases se preserva adecuadamente

3. MODIFICACIONES APLICADAS:
   • {(celdas_modificadas/total_celdas)*100:.2f}% de las celdas fueron modificadas
   • Los cambios se concentran en {len(df_diffs_detalladas)} features específicas
   • Las transformaciones son consistentes y documentadas

4. DATASETS PREPARADOS PARA DVC:
   ✓ v1_original: Baseline sin modificaciones
   ✓ v2_cleaned_aligned: Version limpia comparable (mismo tamaño)
   ✓ v2_cleaned_full: Versión limpia completa para modelado

💡 RECOMENDACIONES:
   • Usar v2_cleaned_full para entrenamiento de modelos
   • Mantener v1_original como baseline de referencia
   • Versionar ambos datasets con DVC para trazabilidad
   • Documentar todas las transformaciones en el pipeline

╚══════════════════════════════════════════════════════════════════════════════╝
"""

print(reporte)

# Guardar reporte en la carpeta correcta de MLOps
report_path = reports_dir / 'turkish_dataset_comparison_report.txt'
with open(report_path, 'w', encoding='utf-8') as f:
    f.write(reporte)

print(f"\n✅ Reporte guardado en: {report_path.resolve()}")
print(f"📊 Ubicación MLOps: reports/turkish_dataset_comparison_report.txt")

---
## 16. Guía para Configuración de DVC
---

---
## 🎯 Resumen Final
---

### ✅ Análisis Completado

Este notebook ha realizado una comparación exhaustiva entre el dataset original y el dataset limpio:

1. **Análisis Estructural**: Comparación de dimensiones, tipos de datos y estructura general
2. **Calidad de Datos**: Evaluación de valores nulos, duplicados y outliers
3. **Distribuciones**: Análisis de clases y distribuciones estadísticas
4. **Diferencias Detalladas**: Análisis celda por celda de las transformaciones
5. **Tests Estadísticos**: Validación de similitud de distribuciones (Kolmogorov-Smirnov)
6. **Correlaciones**: Comparación de relaciones entre features
7. **Métricas Globales**: Resumen cuantitativo de todos los cambios

### 📦 Archivos Generados

- `turkish_music_emotion_v1_original.csv` - Dataset original (baseline)
- `turkish_music_emotion_v2_cleaned_aligned.csv` - Dataset limpio alineado (comparación directa)
- `turkish_music_emotion_v2_cleaned_full.csv` - Dataset limpio completo (para modelado)
- `comparison_report.txt` - Reporte detallado de comparación
- `dvc_setup_guide.sh` - Guía completa para configurar DVC

### 🚀 Próximos Pasos

1. Configurar DVC usando la guía proporcionada
2. Versionar los datasets generados
3. Crear pipelines de experimentación
4. Entrenar modelos con diferentes versiones del dataset
5. Comparar resultados de modelos entre versiones

---